<a href="https://colab.research.google.com/github/sudo-cod/RecipeAI/blob/main/recipe_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth==2025.12.9

  Using cached unsloth-2025.12.9-py3-none-any.whl.metadata (65 kB)
  Using cached unsloth_zoo-2026.1.3-py3-none-any.whl.metadata (32 kB)
  Using cached xformers-0.0.33.post2-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached cut_cross_entropy-25.1.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cusolver_cu12-11.7.3.90-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cusparse_cu12-12.5.8.93-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.8 kB)
INFO: pip is looking at multiple versions of torchvision to determin

LLM Finetuning/Training

In [ ]:
# =========================
# 0. Install dependencies (Colab)
# =========================
# !pip install -q unsloth transformers datasets accelerate bitsandbytes trl

# =========================
# 1. Imports
# =========================
import torch
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from unsloth import FastLanguageModel

# =========================
# 2. Load model
# =========================
model_name = "unsloth/llama-3-8b-bnb-4bit"
max_seq_length = 4096

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto",  # ✅ IMPORTANT: fixes Accelerate device(None) crash
)

# Enable LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
)

# =========================
# 3. Load dataset (JSONL)
# =========================
dataset = load_dataset(
    "json",
    data_files="/content/dataset.jsonl",
    split="train"
)

# =========================
# 4. Prompt formatting
# =========================
def format_prompt(example):
    text = (
        "### Instruction:\n"
        f"{example['instruction']}\n\n"
        "### Input:\n"
        f"{example['input']}\n\n"
        "### Response:\n"
        f"{example['output']}"
    )
    return {"text": text}

dataset = dataset.map(
    format_prompt,
    remove_columns=dataset.column_names,
)

# =========================
# 5. Trainer
# =========================
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        output_dir="./food_recipe_lora",
        dataset_text_field="text",  # ✅ REQUIRED
        max_seq_length=max_seq_length,

        per_device_train_batch_size=2,  # ✅ padding-free needs >=2
        gradient_accumulation_steps=4,
        num_train_epochs=3,

        learning_rate=2e-4,
        fp16=True,

        logging_steps=10,
        save_strategy="epoch",
        report_to="none",

        dataset_num_proc=1,  # avoids psutil issues
    ),
)

# =========================
# 6. Train
# =========================
trainer.train()

# =========================
# 7. Save LoRA adapter
# =========================
model.save_pretrained("food_recipe_lora")
tokenizer.save_pretrained("food_recipe_lora")


/tmp/ipython-input-2940548277.py:12: UserWarning: WARNING: Unsloth should be imported before [trl, transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e59622df-0080-4da6-9cd1-0e927bf55705)')' thrown while requesting HEAD https://huggingface.co/unslothai/repeat/resolve/7c48478c02f84ed89f149b0815cc0216ee831fb0/model.safetensors
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6fb3d863-5dbf-4e9a-8745-840c6fd4c6dd)')' thrown while requesting HEAD https://huggingface.co/unslothai/vram-16/resolve/9703344699da71a2bb9f17e575eb918c8f6cb349/model.safetensors
Retrying in 1s [Retry 1/5].
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


TypeError: SFTConfig.__init__() got an unexpected keyword argument 'max_seq_length'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

1. Youtube Link -> Transcript using API call

In [ ]:
import requests
import json

# Configuration
API_KEY = "sk_ZBvSAZPtoIFOVS7vFbo_9K4Y-Dx8nlM3L1TImnUocbA"
BASE_URL = "https://transcriptapi.com/api/v2"

def get_transcript(video_url, format="text", include_timestamp=False, send_metadata=False):
    """Fetch YouTube video transcript"""

    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    params = {
        "video_url": video_url,
        "format": format,
        "include_timestamp": include_timestamp,
        "send_metadata": send_metadata
    }

    try:
        response = requests.get(
            f"{BASE_URL}/youtube/transcript",
            headers=headers,
            params=params
        )

        # Check for errors
        response.raise_for_status()

        # Parse JSON response
        data = response.json()

        return data

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 402:
            error_data = e.response.json()
            print(f"Payment required: {error_data['detail']['message']}")
            print(f"Action: {error_data['detail']['action_url']}")
        elif e.response.status_code in (408, 429, 503):
            # Retryable errors - implement backoff
            retry_after = e.response.headers.get('Retry-After', '5')
            print(f"Retryable error ({e.response.status_code}). Retry after {retry_after} seconds")
        elif e.response.status_code == 404:
            print("Video not found or has no transcript available")
        else:
            print(f"HTTP error: {e}")
    except Exception as e:
        print(f"Error: {e}")

data = get_transcript("https://www.youtube.com/watch?v=SdHE2zgSaxU", send_metadata = False)
transcript = data['transcript']

2. Use finetuned LLM to turn transcript to recipe summary

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=4096,
    dtype=torch.float16,
    load_in_4bit=True,
)

model.load_adapter("food_recipe_lora")
FastLanguageModel.for_inference(model)

prompt = f"""
Convert this cooking video transcript into a structured recipe.
{transcript}

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=800,
    temperature=0.3,
    top_p=0.9,
    do_sample=True,
)

generated_ids = outputs[0][inputs.input_ids.shape[-1]:]
recipe_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

print(recipe_text)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

**Pan-Seared Ribeye Steak with Garlic Butter and Thyme**

**Ingredients:**
* 1 ribeye steak (1.5-2 inches thick)
* 2 tablespoons high-heat oil (such as vegetable or canola)
* Kosher salt and fresh-cracked black pepper, to taste
* 4 tablespoons (56 grams) unsalted butter
* 2 cloves garlic, lightly crushed and left in skins
* 1 bunch fresh thyme sprigs

**Instructions:**
1. Remove the steak from the refrigerator and let it rest at room temperature for 30 minutes.
2. Pat the steak dry with a paper towel. Season generously on both sides with kosher salt and black pepper.
3. Heat a medium to large skillet over medium-high heat. Add the oil and swirl to coat the pan.
4. When the oil is shimmering and the pan is very hot, add the steak. Sear for 2-3 minutes per side for medium-rare (for a 1.5-inch steak), or until a deep brown crust forms.
5. Flip the steak and sear for an additional 30 seconds to 1 minute. Add the butter, garlic, and thyme to the pan.
6. Baste the steak continuously with the

3. Calculate nutrition and calorie information by going through ingredients list

In [ ]:
import re
import requests

API_KEY = "V0DCm1e1vXpZ54f1GCb5vtQj3CXsaesZ0tHzx5pL"

def extract_ingredients(recipe_text):
    ingredients = []
    capture = False

    for line in recipe_text.splitlines():
        line = line.strip()

        if re.search(r"\bingredients\b", line, re.I):
            capture = True
            continue

        if capture and re.search(r"\b(instructions|method|steps)\b", line, re.I):
            break

        if capture and re.match(r"^[-*•]", line):
            ingredients.append(line.lstrip("-*• ").strip())

    return ingredients

def normalize_ingredient(text):
    text = text.lower()

    # remove parentheses content
    text = re.sub(r"\(.*?\)", "", text)

    # remove quantities + units
    text = re.sub(
        r"\b\d+(\.\d+)?\s*(g|kg|ml|l|tbsp|tsp|tablespoons?|teaspoons?)\b",
        "",
        text
    )

    # remove fractions like 1/2
    text = re.sub(r"\b\d+/\d+\b", "", text)

    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)

    # collapse whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

def lookup_usda(food_name):
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        "query": food_name,
        "api_key": API_KEY,
        "pageSize": 5,
        "dataType": ["Foundation", "SR Legacy"]
    }

    r = requests.get(url, params=params)
    data = r.json()

    if "foods" not in data or not data["foods"]:
        return None

    return data["foods"]

def pick_best_food(query, foods):
    query_tokens = set(query.split())

    best = None
    best_score = 0

    for food in foods:
        name = food["description"].lower()
        tokens = set(name.split())

        score = len(query_tokens & tokens)

        if score > best_score:
            best = food
            best_score = score

    return best

def extract_macros(food):
    if not food or "foodNutrients" not in food:
        return {
            "calories": 0,
            "protein": 0,
            "fat": 0,
            "carbs": 0
        }

    nutrients = {
        n.get("nutrientName"): n.get("value", 0)
        for n in food.get("foodNutrients", [])
    }

    return {
        "calories": nutrients.get("Energy", 0),
        "protein": nutrients.get("Protein", 0),
        "fat": nutrients.get("Total lipid (fat)", 0),
        "carbs": nutrients.get("Carbohydrate, by difference", 0),
    }

def extract_quantity(text):
    text = text.lower()

    # grams
    g = re.search(r"(\d+(?:\.\d+)?)\s*g", text)
    if g:
        return float(g.group(1)), "g"

    # milliliters
    ml = re.search(r"(\d+(?:\.\d+)?)\s*ml", text)
    if ml:
        return float(ml.group(1)), "ml"

    # tablespoons (approx 15g)
    tbsp = re.search(r"(\d+(?:\.\d+)?)\s*tbsp", text)
    if tbsp:
        return float(tbsp.group(1)) * 15, "g"

    # teaspoons (approx 5g)
    tsp = re.search(r"(\d+(?:\.\d+)?)\s*tsp", text)
    if tsp:
        return float(tsp.group(1)) * 5, "g"

    # fallback
    return 100.0, "g"

def scale_macros(macros, weight_g):
    factor = weight_g / 100.0
    return {
        k: round(v * factor, 2)
        for k, v in macros.items()
    }

ZERO_CAL_INGREDIENTS = {"salt", "water", "pepper"}

def is_zero_calorie(ingredient):
    return ingredient.strip() in ZERO_CAL_INGREDIENTS

# --------------------
# Run pipeline
# --------------------

ingredients = extract_ingredients(recipe_text)

results = []

final_ingredients = []
total_macros = {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}

for ing in ingredients:
    normalized = normalize_ingredient(ing)
    qty, unit = extract_quantity(ing)

    foods = lookup_usda(normalized)
    food = pick_best_food(normalized, foods) if foods else None

    if is_zero_calorie(normalized):
        macros = {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}
        scaled = macros
        matched = "water (ignored)"
    else:
        macros = extract_macros(food)
        scaled = scale_macros(macros, qty)
        matched = food["description"] if food else None

    for k in total_macros:
        total_macros[k] += scaled[k]

    final_ingredients.append({
        "original": ing,
        "normalized": normalized,
        "quantity_g": qty,
        "matched_usda_food": matched,
        "macros": scaled
    })

print("\nTotal Macros for Recipe:")
print(f"Calories: {round(total_macros['calories'])} kcal")
print(f"Protein: {round(total_macros['protein'])} g")
print(f"Fat: {round(total_macros['fat'])} g")
print(f"Carbs: {round(total_macros['carbs'])} g")


# per ingredient macros, rounded
print("\nIngredients and Macros:")
for ing in final_ingredients:
     m = ing['macros']
     print(f"- {ing['original']}: Calories {round(m['calories'])} kcal, "
          f"Protein {round(m['protein'])} g, Fat {round(m['fat'])} g, Carbs {round(m['carbs'])} g")




Total Macros for Recipe:
Calories: 2013 kcal
Protein: 38 g
Fat: 62 g
Carbs: 107 g

Ingredients and Macros:
- 1 ribeye steak (1.5-2 inches thick): Calories 784 kcal, Protein 20 g, Fat 11 g, Carbs 2 g
- 2 tablespoons high-heat oil (such as vegetable or canola): Calories 0 kcal, Protein 0 g, Fat 0 g, Carbs 0 g
- Kosher salt and fresh-cracked black pepper, to taste: Calories 1050 kcal, Protein 10 g, Fat 3 g, Carbs 64 g
- 4 tablespoons (56 grams) unsalted butter: Calories 0 kcal, Protein 0 g, Fat 46 g, Carbs 0 g
- 2 cloves garlic, lightly crushed and left in skins: Calories 78 kcal, Protein 3 g, Fat 0 g, Carbs 17 g
- 1 bunch fresh thyme sprigs: Calories 101 kcal, Protein 6 g, Fat 2 g, Carbs 24 g
